In [1]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT
import torch
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)

In [4]:
# import sb3
from stable_baselines3 import PPO

model = PPO("CnnPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
# Train the agent
model.learn(total_timesteps=int(1e5))

In [1]:
from Custom_Tetris_Env.tetris import TetrisEnv
from Custom_Tetris_Env.jp import JoypadSpace
from Custom_Tetris_Env.actions import MOVEMENT
from stable_baselines3 import DQN



env = TetrisEnv()
env = JoypadSpace(env, MOVEMENT)

model = DQN("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/jack/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Train the agent
model.learn(total_timesteps=int(1e6))

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.58e+03 |
|    ep_rew_mean      | -20      |
|    exploration rate | 0.674    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 650      |
|    time_elapsed     | 52       |
|    total timesteps  | 34310    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.11e+03 |
|    ep_rew_mean      | -20      |
|    exploration rate | 0.383    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 628      |
|    time_elapsed     | 103      |
|    total timesteps  | 64905    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000385 |
|    n_updates        | 3726     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

In [7]:
# save model to file
model.save("save_models/ppo_tetris")

In [6]:
done = False
obs = env.reset()


while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()

In [ ]:
piece_dict = {'Tu': 0, 'Tr': 1, 'Td': 2, 'Tl': 3, 'Jl': 4, 'Ju': 5, 'Jr': 6, 'Jd': 7, 'Zh': 8, 'Zv': 9, 'O': 10, 'Sh': 11, 'Sv': 12, 'Lr': 13, 'Ld': 14, 'Ll': 15, 'Lu': 16, 'Iv': 17, 'Ih': 18}

def process_board(board):
    # Returns board, holes and boundaries
    board = np.where(board == 239,0,1)
    holes = np.zeros(board.shape[1], dtype=int)
    boundaries = np.zeros(board.shape[1], dtype=int)

    for col in range(board.shape[1]):
        column_data = board[:, col]
        first_block_idx = np.where(column_data == 1)[0]
        if first_block_idx.size > 0:
            first_block_idx = first_block_idx[0]
            boundaries[col] = board.shape[0] - first_block_idx
            holes[col] = np.count_nonzero(column_data[first_block_idx:] == 0)
    
    return board,holes,boundaries

def concat_data(holes,boundaries,cur_piece,cleared_lines):
    combined_array = np.concatenate((holes, boundaries))
    final_array = np.append(combined_array, [cur_piece, cleared_lines])
    return final_array

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    cur_piece = piece_dict[info["current_piece"]]
    cleared_lines = info["number_of_lines"]
    board, holes, boundaries = process_board(env.board)
    state_data = concat_data(holes,boundaries,cur_piece,cleared_lines)
    state_data = state_data.copy()
    state_data = torch.tensor(state_data, dtype=torch.float32)
    

In [ ]:
state_data

tensor([1., 3., 3., 2., 2., 0., 1., 2., 0., 0., 7., 6., 6., 6., 6., 3., 4., 4.,
        0., 0., 3., 0.])